In [1]:
from jax import jit, vmap, random
import jax.numpy as np
import numpy as onp
import jax

# enable float 64
from jax.config import config
config.update('jax_enable_x64', True)

In [2]:
from utils_response import create_df_response
import pandas as pd
import datetime


In [3]:
# https://github.com/google/jax/issues/10144
def transition(xi, τ, dt=1):
    kb    = np.maximum(1.0 - np.exp(-τ*dt), 0)
    pop   = onp.random.binomial(onp.int64(xi), kb )
    return jax.device_put(pop)

def checkpopvars(x, pop):
    return np.minimum(np.maximum(x, 0), pop)

def f(x, dt, β, γ, N):
    """
    Process model
    """
    S = x.at[0, :].get()
    I = x.at[1, :].get()
    R = x.at[2, :].get()
    C = x.at[3, :].get()

    s2i = transition(S, β * I / N, dt)
    i2r = transition(I, γ, dt)

    S  = S - s2i
    I  = I + s2i - i2r
    R  = R + i2r
    C  = s2i

    return checkpopvars(np.array([S, I, R, C]), N)

def g(x):
    """
    Observational model.
    """
    return x.at[3, :].get()

def f0(pop=1e-6, inf_init=1/100, m =300):
    """
    Initial guess of the state space.
    """

    I0 = pop * 0.01
    S0 = pop - inf_init
    R0 = 0
    C0 = 0

    x0  = np.ones((4, m)) * np.expand_dims(np.array([S0, I0, R0, C0]), -1)

    return x0

In [5]:
N = 1000

β = 0.3 # days
γ = 1/7 # days


T    = 35
dt   = 1
ens  = 500
x0    = f0(N, 1/100, m=ens)
x_sim = x0

x_sim = np.full((4, T, ens), np.nan)
x_sim = x_sim.at[:, 0, :].set(x0)

for t in range(1, T):
    x     = f(x_sim.at[:, t-1, :].get(), 1, β, γ, N)
    x_sim = x_sim.at[:, t, :].set(x)


In [ ]:
S =  x_sim.at[0, :, :].get(); S_df = create_df_response(S.T, T)
I =  x_sim.at[1, :, :].get(); I_df = create_df_response(I.T, T)
R =  x_sim.at[2, :, :].get(); R_df = create_df_response(R.T, T)
C =  x_sim.at[3, :, :].get(); C_df = create_df_response(C.T, T)

k = jax.random.PRNGKey(1)
obs_use = np.maximum(C.at[:, 300].get() + np.squeeze(jax.random.normal(k, shape=(T, 1))*2), 0)


In [ ]:
S_df.index.shape, obs_use.shape, C.at[:, 300].get().shape,  jax.random.normal(k, shape=(T, 1)).shape

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].plot(S_df.index, S_df['mean'], label='S', color="k")
ax[0].fill_between(S_df.index, S_df['low_95'], S_df['high_95'], alpha=0.4, color="gray")
ax[0].fill_between(S_df.index, S_df['low_50'], S_df['high_50'], alpha=0.3, color="gray")

ax[1].plot(C_df.index, C_df['mean'], label='S', color="k")
ax[1].fill_between(C_df.index, C_df['low_95'], C_df['high_95'], alpha=0.4, color="gray")
ax[1].fill_between(C_df.index, C_df['low_50'], C_df['high_50'], alpha=0.3, color="gray")
ax[1].scatter(S_df.index, obs_use, color="red", marker="x", s=20)


In [ ]:
def eakf(x, y_sim, y, oev, return_gain=False):
    """ Ensemble Adjustment Kalman Filter

    Args:
        x    : State space at time t.
        y_sim: Simulated observation.
        y    : Measured observation.
        oev  : Observation error variance.
        return_gain (bool, optional): If true return the gain of the filter. Defaults to False.

    Returns:
        xpost: Posterior estimate of state space.
        obspost: Posterior estimate of observation.
    """
    num_vars, num_ens = x.shape
    prior_mean_ct     = y_sim.mean(-1, keepdims=True)  # Average over ensemble members
    prior_var_ct      = y_sim.var(-1, keepdims=True)   # Compute variance over ensemble members

    post_var_ct   = prior_var_ct * oev / (prior_var_ct + oev)
    post_mean_ct  = post_var_ct * (prior_mean_ct/prior_var_ct + y / oev)
    alpha         = oev / (oev+prior_var_ct); alpha = alpha**0.5
    dy            = post_mean_ct + alpha*( y_sim - prior_mean_ct ) - y_sim

    A       = np.cov(x, y_sim)
    covars  = A.at[:-1, -1].get()
    rr      = covars / prior_var_ct
    dx      = np.dot( np.expand_dims(rr, -1), np.expand_dims(dy, 0))
    xpost   = x     + dx
    obspost = y_sim + dy

    if return_gain:
        return xpost, obspost, dx
    else:
        return xpost, obspost

def geometric_cooling(if_iters, cooling_factor=0.9):
    alphas = cooling_factor**np.arange(if_iters)
    return alphas**2

def hyperbolic_cooling(if_iters, cooling_factor=0.9):
    alphas = 1/(1+cooling_factor*np.arange(if_iters))
    return alphas

def cooling(num_iteration_if, type_cool="geometric", cooling_factor=0.9):
    if type_cool=="geometric":
        return geometric_cooling(num_iteration_if, cooling_factor=cooling_factor)
    elif type_cool=="hyperbolic":
        return hyperbolic_cooling(num_iteration_if, cooling_factor=cooling_factor)
